In [1]:
from pyspark.sql import SparkSession

spark = SparkSession.builder.appName("Recommender_System_Basic").getOrCreate()

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/03/16 11:02:22 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [4]:
from pyspark.ml.recommendation import ALS
from pyspark.ml.evaluation import RegressionEvaluator

In [5]:
data = spark.read.csv("movielens_ratings.csv", inferSchema=True, header=True)

In [6]:
data.show()

+-------+------+------+
|movieId|rating|userId|
+-------+------+------+
|      2|   3.0|     0|
|      3|   1.0|     0|
|      5|   2.0|     0|
|      9|   4.0|     0|
|     11|   1.0|     0|
|     12|   2.0|     0|
|     15|   1.0|     0|
|     17|   1.0|     0|
|     19|   1.0|     0|
|     21|   1.0|     0|
|     23|   1.0|     0|
|     26|   3.0|     0|
|     27|   1.0|     0|
|     28|   1.0|     0|
|     29|   1.0|     0|
|     30|   1.0|     0|
|     31|   1.0|     0|
|     34|   1.0|     0|
|     37|   1.0|     0|
|     41|   2.0|     0|
+-------+------+------+
only showing top 20 rows



In [7]:
data.printSchema()

root
 |-- movieId: integer (nullable = true)
 |-- rating: double (nullable = true)
 |-- userId: integer (nullable = true)



In [8]:
data.describe().show()

+-------+------------------+------------------+------------------+
|summary|           movieId|            rating|            userId|
+-------+------------------+------------------+------------------+
|  count|              1501|              1501|              1501|
|   mean| 49.40572951365756|1.7741505662891406|14.383744170552964|
| stddev|28.937034065088994| 1.187276166124803| 8.591040424293272|
|    min|                 0|               1.0|                 0|
|    max|                99|               5.0|                29|
+-------+------------------+------------------+------------------+



In [9]:
train_data, test_data = data.randomSplit([0.7, 0.3], seed=20)

In [10]:
als = ALS(
    maxIter=5, regParam=0.01, userCol="userId", itemCol="movieId", ratingCol="rating"
)

In [11]:
model = als.fit(train_data)

24/03/16 11:07:16 WARN InstanceBuilder: Failed to load implementation from:dev.ludovic.netlib.blas.JNIBLAS
24/03/16 11:07:16 WARN InstanceBuilder: Failed to load implementation from:dev.ludovic.netlib.blas.VectorBLAS
24/03/16 11:07:17 WARN InstanceBuilder: Failed to load implementation from:dev.ludovic.netlib.lapack.JNILAPACK


In [13]:
predictions = model.transform(test_data)

In [14]:
predictions.show()

+-------+------+------+----------+
|movieId|rating|userId|prediction|
+-------+------+------+----------+
|      1|   1.0|    28| 1.0680226|
|      1|   1.0|    26| 1.4877014|
|      5|   2.0|    22|0.89205647|
|      4|   2.0|     1| 2.5846648|
|      5|   1.0|    13| 0.6580615|
|      0|   1.0|     6| 1.8457111|
|      2|   3.0|     6| 2.6484213|
|      5|   3.0|    16| 1.6890459|
|      1|   1.0|     3| 0.5305238|
|      5|   1.0|     5| 1.1468563|
|      4|   1.0|    19| 1.3454784|
|      2|   1.0|    15| 1.3852539|
|      1|   1.0|     4|0.53958356|
|      0|   1.0|    23|0.63166976|
|      4|   1.0|    23|0.04209382|
|      2|   4.0|    10| 3.4964554|
|      5|   1.0|    29|  1.034793|
|      2|   4.0|    21| 1.7807906|
|      6|   1.0|     2|-0.7796788|
|      2|   3.0|     0| 2.4797668|
+-------+------+------+----------+
only showing top 20 rows



In [17]:
evaluator = RegressionEvaluator(
    metricName="rmse", labelCol="rating", predictionCol="prediction"
)

In [18]:
rmse = evaluator.evaluate(predictions)

In [19]:
print("RMSE")
print(rmse)

RMSE
1.937408235252996


In [20]:
single_user = test_data.filter(test_data["userId"] == 11).select("movieId", "userId")

In [21]:
single_user.show()

+-------+------+
|movieId|userId|
+-------+------+
|      9|    11|
|     11|    11|
|     12|    11|
|     30|    11|
|     62|    11|
|     66|    11|
|     71|    11|
|     76|    11|
|     86|    11|
+-------+------+



In [22]:
recommendations = model.transform(single_user)

In [23]:
recommendations.orderBy("prediction", ascending=False).show()

+-------+------+----------+
|movieId|userId|prediction|
+-------+------+----------+
|     62|    11| 2.7655735|
|     12|    11| 1.8881475|
|     66|    11|0.79136974|
|     71|    11|0.38149127|
|      9|    11|0.31212756|
|     11|    11|0.30042106|
|     30|    11|-1.0469849|
|     86|    11|-1.2149444|
|     76|    11| -2.228359|
+-------+------+----------+

